In [ ]:
# Importando as livrarias necessárias
import pandas as pd
import os
import py7zr
import tempfile
import urllib

In [ ]:
# Definindo uma função que lê e extrai os arquivos do servidor ftp e armazena em um DataFrame pandas
def extract_rais_data(url, nrows=None, cols=None):
    """
    Lê, descompacta e retorna os dados de ftp server em um DataFrame pandas.
    
    A função acessa e extrai os dados do RAIS direto do servidor ftp do Ministério do Trabalho.
    Armazena temporariamente os dados em diretório local, descompacta — arquivo em formato 7z — para posteriormente 
    disponibilizar em um DataFrame pandas para análises futuras.

    Parâmetros:
        url: (link). Recebe o caminho do arquivo.
        nrows: (int), value_default=None. Número de linhas do dataset a set importado.
        cols: (list-like), value_default=None. Recebe colunas específicas a serem importadas.
    
    Retorna: 
        pandas.DataFrame. Retorna um DataFrame pandas.
    """
    
    # Lê os dados do servidor ftp
    with urllib.request.urlopen(url) as file:
        file_content = file.read()
    
    # Cria um diretório temporário local para armazenar os dados
    with tempfile.TemporaryDirectory() as temp_dir:
        path_file_7z = os.path.join(temp_dir, "temp_file.7z")
        with open(path_file_7z, mode="wb") as f:
            f.write(file_content)

        # Descompacta os arquivos 7z
        with py7zr.SevenZipFile(path_file_7z, mode="r") as z:
            z.extractall(path=temp_dir)

        # Armazena os arquivos descompactados em uma variável `csv_path`
        extracted_file = os.listdir(temp_dir)
        csv_path = None

        # Itera sobre os arquivos descompactados em busca de arquivo csv
        for file_name in extracted_file:
            full_path = os.path.join(temp_dir, file_name)

            try:
                pd.read_csv(full_path, sep=";", nrows=5, encoding="ISO-8859-1")
                csv_path = full_path
                break
            except Exception:
                continue
        
        if not csv_path:
            raise FileNotFoundError("Nenhum arquivo csv foi encontrado no arquivo 7z.")

        # Lê os dados em um DataFrame com diferentes tentativas de condificações
        try:
            df = pd.read_csv(csv_path, sep=";", header=0, encoding="UTF-8", nrows=nrows, usecols=cols)
        
        except UnicodeDecodeError:
            df = pd.read_csv(csv_path, sep=";", header=0, encoding="ISO-8859-1", nrows=nrows, usecols=cols)
        
    return df

In [ ]:
# Caminho do arquivo RAIS no servidor ftp
path = "ftp://ftp.mtps.gov.br/pdet/microdados/RAIS/2024/RAIS_VINC_PUB_SP.7z"

# Chama a função e passa os argumentos
df = extract_rais_data(url=path, nrows=200)

In [ ]:
# Explorando as variáveis do conjunto de dados
df.info(memory_usage="deep")